In [0]:
# fetch parameters
par_matchup_week = int( dbutils.widgets.get("MATCHUP_WEEK") )
par_year = dbutils.widgets.get("YEAR")
par_lookback_hours = dbutils.widgets.get('TREND_LOOKBACK_HOURS')
par_lookback_limit = dbutils.widgets.get('TREND_LOOKBACK_LIMIT')

In [0]:
%run /Workspace/Users/djamsheedy99@gmail.com/SleeperWarehouse/src/ingest/_utils_sleeper

In [0]:
ingest_util = SleeperIngestUtil(year = par_year, matchup_week = par_matchup_week)

## v1/players

In [0]:
if par_matchup_week > 0:
    player_api_response = requests.get(f'https://api.sleeper.app/v1/players/nfl')
    player_api_raw_json = json.loads(player_api_response.text)

    data = [v for k, v in player_api_raw_json.items() if isinstance(v, dict) and 'player_id' in v.keys()]

    data = ingest_util.add_metadata(data)

    endpoint_name = 'players'
    ingest_util.write_data(data, endpoint_name)
    # dbutils.fs.put(f"{tgt_dir}/players/{par_year}/players_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(player_api_json), overwrite=True)

    trend_types = ['add', 'drop']

    for trend_type in trend_types:
        data = ingest_util.read_api(f'https://api.sleeper.app/v1/players/nfl/trending/{trend_type}?lookback_hours={par_lookback_hours}&limit={par_lookback_limit}')
        data = ingest_util.add_metadata(data)
        ingest_util.write_data(data, f'{endpoint_name}_trend_{trend_type}')
